In [1]:
import pandas as pd
import numpy as np
from IPython.display import Image

In [2]:
pd.__version__

'1.4.2'

# Intro

Pandas' groupby is undoubtly one of the most powerful functionalities that Pandas brings to the table. However, most users only utilize a fraction of the capabilities of `groupby`. 

`Groupby` allows to adopt a split-apply-combine approach to your data set. This is comparable to slicing and dicing your data such that it serves your specific need.

On a high level this means:
1. split the data based on column(s)/condition(s) into groups
2. apply a function/transformation to all the groups and combine the results into an output

# 0. Load the data

We are going to use data from a hypothetical sales division where we have, among other columns ficticious sales reps, order leads, order values, the company the deal might happen with and the date of the order lead.

In [3]:
order_leads = pd.read_csv(
    'https://raw.githubusercontent.com/FBosler/Medium-Data-Exploration/master/order_leads.csv',
    parse_dates = [3]
)
sales_team = pd.read_csv(
    'https://raw.githubusercontent.com/FBosler/Medium-Data-Exploration/master/sales_team.csv',
    
)


In [4]:
order_leads.head()

Order Id        Company Id            Company Name       Date  \
0  HZSXLI1IS9RGABZW  D0AUXPP07H6AVSGD  Melancholy Social-Role 2017-10-13   
1  582WPS3OW8T6YT0R  D0AUXPP07H6AVSGD  Melancholy Social-Role 2017-09-02   
2  KRF65MQZBOYG4Y9T  D0AUXPP07H6AVSGD  Melancholy Social-Role 2016-12-21   
3  N3EDZ5V1WGSWW828  D0AUXPP07H6AVSGD  Melancholy Social-Role 2018-06-03   
4  QXBC8COXEXGFSPLP  D0AUXPP07H6AVSGD  Melancholy Social-Role 2014-07-26   

   Order Value  Converted  
0         6952          0  
1         7930          0  
2         5538          1  
3         1113          0  
4         4596          0

In [5]:
order_leads.shape

(100000, 6)

In [6]:
order_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Order Id      100000 non-null  object        
 1   Company Id    100000 non-null  object        
 2   Company Name  100000 non-null  object        
 3   Date          100000 non-null  datetime64[ns]
 4   Order Value   100000 non-null  int64         
 5   Converted     100000 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 4.6+ MB


In [7]:
sales_team.head()

Sales Rep      Sales Rep Id            Company Name        Company Id
0  William Taylor  ZTZA0ZLYZR85PTUJ  Melancholy Social-Role  D0AUXPP07H6AVSGD
1  William Taylor  ZTZA0ZLYZR85PTUJ          Pee-Wee Pigeon  TJW99R4RWH02INII
2  William Taylor  ZTZA0ZLYZR85PTUJ     Vaginal 36-Year-Old  3S20IO1O1DYXMUNU
3  William Taylor  ZTZA0ZLYZR85PTUJ    Unhurried Musician'S  FDWW0TQRPGMVI2A7
4  William Taylor  ZTZA0ZLYZR85PTUJ     Comprehensive Lists  C30FY83UNLVBA0O3

In [8]:
sales_team.shape

(4619, 4)

In [9]:
order_leads.columns

Index(['Order Id', 'Company Id', 'Company Name', 'Date', 'Order Value',
       'Converted'],
      dtype='object')

In [10]:
sales_team.columns

Index(['Sales Rep', 'Sales Rep Id', 'Company Name', 'Company Id'], dtype='object')

In [11]:
df = pd.merge(order_leads,sales_team,on=['Company Id','Company Name'])
df.head()

Order Id        Company Id            Company Name       Date  \
0  HZSXLI1IS9RGABZW  D0AUXPP07H6AVSGD  Melancholy Social-Role 2017-10-13   
1  582WPS3OW8T6YT0R  D0AUXPP07H6AVSGD  Melancholy Social-Role 2017-09-02   
2  KRF65MQZBOYG4Y9T  D0AUXPP07H6AVSGD  Melancholy Social-Role 2016-12-21   
3  N3EDZ5V1WGSWW828  D0AUXPP07H6AVSGD  Melancholy Social-Role 2018-06-03   
4  QXBC8COXEXGFSPLP  D0AUXPP07H6AVSGD  Melancholy Social-Role 2014-07-26   

   Order Value  Converted       Sales Rep      Sales Rep Id  
0         6952          0  William Taylor  ZTZA0ZLYZR85PTUJ  
1         7930          0  William Taylor  ZTZA0ZLYZR85PTUJ  
2         5538          1  William Taylor  ZTZA0ZLYZR85PTUJ  
3         1113          0  William Taylor  ZTZA0ZLYZR85PTUJ  
4         4596          0  William Taylor  ZTZA0ZLYZR85PTUJ

In [12]:
df.shape

(100000, 8)

In [13]:
# Select sales complete
df[df.Converted==1]

Order Id        Company Id                    Company Name  \
2      KRF65MQZBOYG4Y9T  D0AUXPP07H6AVSGD          Melancholy Social-Role   
7      JFMNRRRNS5MD8BKQ  D0AUXPP07H6AVSGD          Melancholy Social-Role   
13     TD5F4GJ2LPIJMIM7  D0AUXPP07H6AVSGD          Melancholy Social-Role   
14     A96YQNYAU7Z9AM70  D0AUXPP07H6AVSGD          Melancholy Social-Role   
16     5WNCKI809TG9M568  D0AUXPP07H6AVSGD          Melancholy Social-Role   
...                 ...               ...                             ...   
99986  FLLDB4PNB0Z3CSCM  APH243SK72T90MPS  Trade-Preparatory Quarterbacks   
99990  I2M8DEJSX9D8956T  APH243SK72T90MPS  Trade-Preparatory Quarterbacks   
99992  VDI6DV3GWK1JCAJM  APH243SK72T90MPS  Trade-Preparatory Quarterbacks   
99994  L95YJURYBYPKLS5Q  APH243SK72T90MPS  Trade-Preparatory Quarterbacks   
99996  962CSDMAJ49E0CRK  APH243SK72T90MPS  Trade-Preparatory Quarterbacks   

            Date  Order Value  Converted       Sales Rep      Sales Rep Id  
2     2016-12-21         5538          1  William Taylor  ZTZA0ZLYZR85PTUJ  
7     2018-02-27         8349          1  William Taylor  ZTZA0ZLYZR85PTUJ  
13    2018-07-03          913          1  William Taylor  ZTZA0ZLYZR85PTUJ  
14    2015-07-05          516          1  William Taylor  ZTZA0ZLYZR85PTUJ  
16    2017-07-09         5370          1  William Taylor  ZTZA0ZLYZR85PTUJ  
...          ...          ...        ...             ...               ...  
99986 2017-06-09         1293          1    Ida Woodward  LF3CPWWZKSNB1AXI  
99990 2014-11-16         9539          1    Ida Woodward  LF3CPWWZKSNB1AXI  
99992 2017-04-18          178          1    Ida Woodward  LF3CPWWZKSNB1AXI  
99994 2018-05-19         9359          1    Ida Woodward  LF3CPWWZKSNB1AXI  
99996 2018-08-02          442          1    Ida Woodward  LF3CPWWZKSNB1AXI  

[16908 rows x 8 columns]

In [14]:
df = df.rename(columns={'Order Value':'Val','Converted':'Sale'})
df

Order Id        Company Id                    Company Name  \
0      HZSXLI1IS9RGABZW  D0AUXPP07H6AVSGD          Melancholy Social-Role   
1      582WPS3OW8T6YT0R  D0AUXPP07H6AVSGD          Melancholy Social-Role   
2      KRF65MQZBOYG4Y9T  D0AUXPP07H6AVSGD          Melancholy Social-Role   
3      N3EDZ5V1WGSWW828  D0AUXPP07H6AVSGD          Melancholy Social-Role   
4      QXBC8COXEXGFSPLP  D0AUXPP07H6AVSGD          Melancholy Social-Role   
...                 ...               ...                             ...   
99995  HKZFX556ZQRZJZWR  APH243SK72T90MPS  Trade-Preparatory Quarterbacks   
99996  962CSDMAJ49E0CRK  APH243SK72T90MPS  Trade-Preparatory Quarterbacks   
99997  ZW7RO9TLL6EVVJEC  APH243SK72T90MPS  Trade-Preparatory Quarterbacks   
99998  LNKGIWMZ9RT49IE9  APH243SK72T90MPS  Trade-Preparatory Quarterbacks   
99999  X9Y21H4JWX6OGC2Z  APH243SK72T90MPS  Trade-Preparatory Quarterbacks   

            Date   Val  Sale       Sales Rep      Sales Rep Id  
0     2017-10-13  6952     0  William Taylor  ZTZA0ZLYZR85PTUJ  
1     2017-09-02  7930     0  William Taylor  ZTZA0ZLYZR85PTUJ  
2     2016-12-21  5538     1  William Taylor  ZTZA0ZLYZR85PTUJ  
3     2018-06-03  1113     0  William Taylor  ZTZA0ZLYZR85PTUJ  
4     2014-07-26  4596     0  William Taylor  ZTZA0ZLYZR85PTUJ  
...          ...   ...   ...             ...               ...  
99995 2017-11-06  7516     0    Ida Woodward  LF3CPWWZKSNB1AXI  
99996 2018-08-02   442     1    Ida Woodward  LF3CPWWZKSNB1AXI  
99997 2014-11-02  8544     0    Ida Woodward  LF3CPWWZKSNB1AXI  
99998 2017-04-01  6650     0    Ida Woodward  LF3CPWWZKSNB1AXI  
99999 2016-07-27   953     0    Ida Woodward  LF3CPWWZKSNB1AXI  

[100000 rows x 8 columns]

# 1. Groupby: Split the data based on condition/column into groups

The default approach of calling groupby, is by explicitly providing a column name to split the dataset by. However, and this is less known, you can also pass a Series (has to have the same length as the dataframe) to groupby. 
This means that you can group by a processed version of a column, without having to create a new helper column for that.

### groupby sales rep
First let's create a grouped DataFrame, i.e. split the dataset up.

In [15]:
grouped_by_sales_rep = df.groupby('Sales Rep')
grouped_by_sales_rep

### show all groups
calling `groups` on the grouped object returns the list of indices for every group (as every row can be uniquely identified via it's index)

In [16]:
grouped_by_sales_rep.groups

{'Aaron Hendrickson': [25612, 25613, 25614, 25615, 25616, 25617, 25618, 25619, 25620, 25621, 25622, 25623, 25624, 25625, 25626, 25627, 25628, 25629, 25630, 25631, 25632, 25633, 25634, 25635, 25636, 25637, 25638, 25639, 25640, 25641, 25642, 25643, 25644, 25645, 25646, 25647, 25648, 25649, 25650, 25651, 25652, 25653, 25654, 25655, 25656, 25657, 25658, 25659, 25660, 25661, 25662, 25663, 25664, 25665, 25666, 25667, 25668, 25669, 25670, 25671, 25672, 25673, 25674, 25675, 25676, 25677, 25678, 25679, 25680, 25681, 25682, 25683, 25684, 25685, 25686, 25687, 25688, 25689, 25690, 25691, 25692, 25693, 25694, 25695, 25696, 25697, 25698, 25699, 25700, 25701, 25702, 25703, 25704, 25705, 25706, 25707, 25708, 25709, 25710, 25711, ...], 'Adam Sawyer': [67140, 67141, 67142, 67143, 67144, 67145, 67146, 67147, 67148, 67149, 67150, 67151, 67152, 67153, 67154, 67155, 67156, 67157, 67158, 67159, 67160, 67161, 67162, 67163, 67164, 67165, 67166, 67167, 67168, 67169, 67170, 67171, 67172, 67173, 67174, 67175, 671

In [17]:
len(grouped_by_sales_rep.groups)

499

### select a specific group

In [18]:
grouped_by_sales_rep.get_group('Aaron Hendrickson')

Order Id        Company Id        Company Name       Date  \
25612  3BJY12LWBN7D0GJL  CE4544HJOFMONMH2  Follow-Up Boundary 2014-09-04   
25613  W3HHOSC1H6A1PW37  CE4544HJOFMONMH2  Follow-Up Boundary 2015-09-24   
25614  G9JKIZO4WD945GBH  CE4544HJOFMONMH2  Follow-Up Boundary 2014-12-06   
25615  BKIJVKZ7REVN6P8B  CE4544HJOFMONMH2  Follow-Up Boundary 2017-05-07   
25616  WFHGWR4PAD04A2GJ  CE4544HJOFMONMH2  Follow-Up Boundary 2016-01-20   
...                 ...               ...                 ...        ...   
25899  NATK7K3TZUH32BBE  CGDGXAW6GNU6JIEG      Fiftieth Art'S 2015-01-27   
25900  EGD6IRB0UML62XB0  CGDGXAW6GNU6JIEG      Fiftieth Art'S 2018-11-04   
25901  9Z18A7D1T8EUH58D  CGDGXAW6GNU6JIEG      Fiftieth Art'S 2016-05-08   
25902  R0LUW64V2F3O2HSD  CGDGXAW6GNU6JIEG      Fiftieth Art'S 2017-02-16   
25903  UMHMBM5M179IHX6D  CGDGXAW6GNU6JIEG      Fiftieth Art'S 2017-07-28   

        Val  Sale          Sales Rep      Sales Rep Id  
25612  1940     0  Aaron Hendrickson  AEMLQ09IYM72ACBL  
25613  2109     0  Aaron Hendrickson  AEMLQ09IYM72ACBL  
25614  4300     1  Aaron Hendrickson  AEMLQ09IYM72ACBL  
25615  3026     0  Aaron Hendrickson  AEMLQ09IYM72ACBL  
25616  5033     0  Aaron Hendrickson  AEMLQ09IYM72ACBL  
...     ...   ...                ...               ...  
25899  6095     1  Aaron Hendrickson  AEMLQ09IYM72ACBL  
25900  7652     1  Aaron Hendrickson  AEMLQ09IYM72ACBL  
25901  4746     0  Aaron Hendrickson  AEMLQ09IYM72ACBL  
25902  6158     0  Aaron Hendrickson  AEMLQ09IYM72ACBL  
25903  2164     0  Aaron Hendrickson  AEMLQ09IYM72ACBL  

[292 rows x 8 columns]

For the following examples we will use the simplest version of the apply step (and just count the rows in each group) via the `size` method. We do this, so that we can focus on the groupby operations. 

We will go into much more detail regarding the apply methods in section 2 of the article. 

### Basic Example: Count rows in each group

In [19]:
grouped_by_sales_rep.size().sort_values(ascending=False)


Sales Rep
Jessica Renner      430
Alyssa Mckinney     428
Elizabeth Gaddy     416
Susan Gutierrez     415
Georgiann White     413
                   ... 
Gloria Perry         18
Alfredo Weise        17
Debra Vermillion     17
Simon Hearn          15
Robert Bowen         15
Length: 499, dtype: int64

In [20]:
df['Sales Rep'].value_counts()

Jessica Renner      430
Alyssa Mckinney     428
Elizabeth Gaddy     416
Susan Gutierrez     415
Georgiann White     413
                   ... 
Viola Ates           18
Debra Vermillion     17
Alfredo Weise        17
Simon Hearn          15
Robert Bowen         15
Name: Sales Rep, Length: 499, dtype: int64

### Advanced Example: Group by first name of sales rep

In [21]:
# Grouping by first name of our Sales Rep without creating a column
df.groupby(df['Sales Rep'].str.split(' ').str[0]).size()

Sales Rep
Aaron        292
Adam         324
Adele        115
Adrian       369
Adrianna      37
            ... 
Wesley       144
Wilbert      213
William     1393
Willie       448
Yvonne       141
Length: 318, dtype: int64

### Advanced Example:  Grouping by whether or not there is a "William" in the name of the rep

In [22]:
df.groupby(df['Sales Rep'].apply(lambda x: 'William' in x)).size()

Sales Rep
False    97111
True      2889
dtype: int64

### Advanced Example: Group by random series (for illustrative purposes only)

In [23]:
# Grouping random letters (obviously does not make sense)
df.groupby(
    pd.Series(np.random.choice(list('ABCDG'),len(df)))
).size()

A    19902
B    19952
C    20267
D    19914
G    19965
dtype: int64

### Advanced Example:  Grouping by 3 evenly cut "Order Value" buckets

In [24]:
# qcut bins the passed Series into q evenly sized parts (and labels the bins)
df.groupby(
    pd.qcut(x=df['Val'],q=3,labels=['low','mid','high'])
).size()

Val
low     33339
mid     33336
high    33325
dtype: int64

### Advanced Example: Grouping by cutom "Order Value" buckets

In [25]:
# cut bins the passed Series into defined bins
df.groupby(
    pd.cut(df['Val'],[0,3000,5000,7000,10000])
).size()

Val
(0, 3000]        29220
(3000, 5000]     19892
(5000, 7000]     20359
(7000, 10000]    30529
dtype: int64

## `pd.Grouper`
The biggest "gotcha" in that area for me was understanding pd.Grouper, which allows seamless aggregation on different date/time granularities.


### Advanced Example: Grouping by year

In [26]:
df.head()

Order Id        Company Id            Company Name       Date  \
0  HZSXLI1IS9RGABZW  D0AUXPP07H6AVSGD  Melancholy Social-Role 2017-10-13   
1  582WPS3OW8T6YT0R  D0AUXPP07H6AVSGD  Melancholy Social-Role 2017-09-02   
2  KRF65MQZBOYG4Y9T  D0AUXPP07H6AVSGD  Melancholy Social-Role 2016-12-21   
3  N3EDZ5V1WGSWW828  D0AUXPP07H6AVSGD  Melancholy Social-Role 2018-06-03   
4  QXBC8COXEXGFSPLP  D0AUXPP07H6AVSGD  Melancholy Social-Role 2014-07-26   

    Val  Sale       Sales Rep      Sales Rep Id  
0  6952     0  William Taylor  ZTZA0ZLYZR85PTUJ  
1  7930     0  William Taylor  ZTZA0ZLYZR85PTUJ  
2  5538     1  William Taylor  ZTZA0ZLYZR85PTUJ  
3  1113     0  William Taylor  ZTZA0ZLYZR85PTUJ  
4  4596     0  William Taylor  ZTZA0ZLYZR85PTUJ

In [27]:
df.groupby(pd.Grouper(key='Date',freq=Y')).Val.agg('max')

SyntaxError: invalid syntax (2120482750.py, line 1)

### Advanced Example: Grouping by quarter

In [ ]:
# grouping by year
df.groupby(pd.Grouper(key='Date',freq='SM')).size()

### Advanced Example: Grouping by multiple columns

In [ ]:
df.groupby(['Sales Rep','Company Name']).size()

# 2. Apply and Combine: Apply a function/transformation to all groups and combine the results into an output

In the previous section we discussed how to group the data based on various conditions. This section deals with available functions that we can apply to the groups before combining them to a final result.

The section is structured along how to use
1. apply,
2. agg(regate),
3. transform, and
4. filter,
on a grouped object.

If you are anything like me when I started using groupby, you are probably using a combination of 1. and 2. along the lines of:

`group = df.groupby('GROUP')` and then:
- `group.apply(mean)`
- `group.agg(mean)`
- `group['INTERSTING COLUMN'].apply(mean)`
- `group.agg({'INTERSTING COLUMN':mean})`
- `group.mean()`

Where `mean` could also be another function.

All of them work. And most of the time, the result is going to be roughly what you expected it to be. However, there are nuances to apply and agg that are worthwhile pointing out.

Additionally, but much more importantly there are two lesser-known extremly powerful functions that can be used on a grouped object, `filter` and `transform`.

### `Apply`: Let's get  `apply` out of the way
This is somewhat confusing, as we often talk about applying functions while there also is an apply function. But bear with me. The `apply` function applies a function along an axis of the DataFrame. This could be either column-wise or row-wise.
`apply` is not strictly speaking a function that can only be used in the context of groupby. It could also used on an entire dataframe, like in the following example.

In [ ]:
_ = pd.DataFrame(
    np.random.random((2,6)),
    columns=list('ABCDEF')
)
_

In [ ]:
_.apply(sum, axis=0) # axis=0 is default, so you can drop that

In [ ]:
_.apply(sum, axis=1)

But it can also be used in a groupby context. Which makes sense, considering the fact that each group is a smaller DataFrame on its own. Keep in mind that the function will be applied to the entire DataFrame. This means typically you want to select the columns you are `applying` a function to. We will leave it at these examples and instead focus on `agg(regation)` which is the "intended" way of aggregating groups.

In [ ]:
df.groupby(
    pd.Grouper(key='Date',freq='Y')
)['Sale'].apply(sum)

In [ ]:
df.groupby(
    pd.Grouper(key='Date',freq='Y')
)['Val','Sale'].apply(sum)

In [ ]:
df.groupby(
    pd.Grouper(key='Date',freq='Y')
)['Val','Sale'].apply(sum).to_clipboard(sep=' ')

# `agg(regate)`

In [ ]:
Image(filename='groupby-agg.png') 

In [ ]:
# agg(regation)
df.groupby('Sales Rep')\
\
.agg({
    'Order Id':'size',
    'Val':['sum','mean'],
    'Sale':['sum','mean']
})

In [ ]:
def cr(x):
    return round(np.mean(x),2)

aggregation = {
    'Potential Sales':('Val','size'),
    'Sales':('Sale','sum'),
    'Conversion Rate':('Sale',cr)
}

df.groupby('Sales Rep').agg(**aggregation)

In [ ]:
def cr(x):
    return round(np.mean(x),2)

_ = df.groupby('Sales Rep').agg({
    'Val':'size',
    'Sale':['sum',cr]
})

_.columns = ['Potential Sales','Sales','Conversion Rate']
_

## Transform

In [ ]:
Image(filename='groupby-transform.png') 

In [ ]:
df.groupby('Sales Rep')['Val'].transform(lambda x: x/sum(x))

In [ ]:
df['%'] = df.groupby('Sales Rep')['Val'].transform(
  lambda x: x/sum(x)
)

In [ ]:
del df['%']

In [ ]:
# filter (at least 200k in sales)
df.groupby('Sales Rep').filter(lambda x: (x['Val'] * x['Sale']).sum() > 200000)

In [ ]:
# Let's add this for verification
df['cr'] = df.groupby('Sales Rep')['Sale'].transform('mean')

df.groupby('Sales Rep').filter(lambda x: x['Sale'].mean() > .3)

In [ ]:
del df['cr']

# Advanced Examples Agg

In [ ]:
# Grouping by 3 evenly cut "Order Value" buckets
df.groupby(
    pd.qcut(df['Val'],3,['low','mid','high'])
).agg({'Val':['mean','std'],'Sale':['sum','size']})

In [ ]:
# Grouping by cutom "Order Value" buckets
df.groupby(
    pd.cut(df['Val'],[0,3000,5000,7000,10000])
).agg({'Val':['mean','std'],'Sale':['sum','size']})

In [ ]:
df.groupby(
    pd.cut(df['Val'],[0,5000,10000],labels=['low','high'])
).agg(
    **{'Conversion Rate':pd.NamedAgg(column='Sale',aggfunc=lambda x: sum(x)/len(x))}
)

In [ ]:
# grouping by year and a cut oder value column
df.groupby(
    [pd.Grouper(key='Date',freq='Y'),pd.qcut(df['Val'],3,['low','mid','high'])]
).agg(
    **{'Conversion Rate':pd.NamedAgg(column='Sale',aggfunc=lambda x: sum(x)/len(x))}
).unstack()